In [1]:
from math import exp, pi, sqrt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker

from scipy.integrate import quad
from scipy.special import gamma
from scipy.optimize import minimize
import scipy.stats as sps
from tqdm import tqdm

In [2]:
sample = [5, 8, 6, 12, 14, 18, 11, 6, 13, 7]
n = 100

In [3]:
I = quad(sps.chi2(9).pdf, 16.4, float('inf'))
I # p-value

(0.05898403054441968, 2.513395414959647e-10)

### Проверка гипотезы о согласии данных с законом равномерного распредления с помощью критерия $χ^2$ и с помощью критерия Колмогорова

In [4]:
def calculate_edf(sample, n):
    frequencies = np.array(sample) / 100
    edf = np.array([np.sum(frequencies[:i + 1]) for i in range(len(sample))])
    return edf

def uniform_density(a, b, x): # равномерное
    return (x - a) / (b - a)

def kolmogorov_density(x): # колмогоровское
    sum_result = 0
    for i in range(1, 10000):
        sum_result += (-1) ** i * exp(-2 * i ** 2 * x ** 2)
    return 1 + 2 * sum_result

def calculate_delta(sample, n, edf):
    previous_height = 0
    delta = -1
    for i in range(len(sample)):
        density_value = uniform_density(0, 9, i)
        delta = max(abs(density_value - edf[i]), abs(density_value - previous_height), delta)
        previous_height = edf[i]
    delta *= sqrt(n)
    return delta

In [5]:
edf = calculate_edf(sample, n)
print(edf)

delta_kolm = calculate_delta(sample, n, edf) #
print('Дельта критерия Колмогорова:', delta_kolm)

print('p-value критерия Колмогорова:', 1 - kolmogorov_density(delta_kolm))

[0.05 0.13 0.19 0.31 0.45 0.63 0.74 0.8  0.93 1.  ]
Дельта критерия Колмогорова: 1.4333333333333331
p-value критерия Колмогорова: 0.032851885438597406


## Проверка гипотезы о согласии данных с законом нормального распределения с помощью критерий $χ^2$ и с помощью критерия Колмогорова

In [6]:
def normal_density(x, tetha_1, tetha_2):
    return 1 / (sqrt(2 * pi) *  tetha_2) * exp(-((x - tetha_1) ** 2) / ((tetha_2 ** 2) * 2))


def normal_function(x, tetha_1, tetha_2):
    return quad(normal_density, x[0], x[1], args=(tetha_1, tetha_2))[0]

def likelihood_function(parameters, sample):
    mean, sigma = parameters
    intervals = [[-float('inf'), 1]] + [[i + 1, i + 2] for i in range(8)] + [[9, float('inf')]]

    func = 1
    for i in range(len(intervals)):
        func *= normal_function(intervals[i], mean, sigma) ** sample[i]
    
    return -func  # я сомневаюсь в этом, но предположим что максимум и минимум поменяются местами

def find_parameters(sample):
    start_point = [3, 2]
    result = minimize(likelihood_function, start_point, args=(sample), method='Nelder-Mead')
    mean, sigma = result.x
    return mean, sigma

In [7]:
mean, sigma = find_parameters(sample)
print('Оценки параметров ОМПГ: cреднее =', mean, 'отклонение =', sigma)


intervals = [[-float('inf'), 1]] + [[i + 1, i + 2] for i in range(8)] + [[9, float('inf')]]
delta = 0

for i in range(len(sample)):
    np_ = n * normal_function(intervals[i], mean, sigma)
    delta += (sample[i] - np_) ** 2 / np_
print('Дельта для критерия χ²:', delta)


p_value_ompg = quad(sps.chi2(7).pdf, delta, float('inf'))[0]
print('p-value критерия χ²:', p_value_ompg)

Оценки параметров ОМПГ: cреднее = 5.289737726548878 отклонение = 2.6795139919364033
Дельта для критерия χ²: 9.802444291486268
p-value критерия χ²: 0.20004793606221866


In [9]:
N = 50000


def calculate_sample(bootstrap_sample):
    borders =  [-float('inf'), 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, float('inf')]
    sample = []
    for i in range(len(borders) - 1):
        a = bootstrap_sample[bootstrap_sample < borders[i + 1]]
        a = a[a > borders[i]]
        sample.append(len(a))
    return sample


def calculate_delta_bootstrap(mean, sigma, edf):
    previous_height = 0
    delta = -1
    for i in range(len(intervals)):
        density_value = sps.norm.cdf(i, loc=mean, scale=sigma)
        delta = max(abs(density_value - edf[i]), abs(density_value - previous_height), delta)
        previous_height = edf[i]
    delta *= sqrt(n)
    return delta

In [12]:
deltas = []

k = 0
pbar = tqdm(range(N))
for i in pbar:
    bsample = sps.norm(loc=mean, scale=sigma).rvs(100)
    csample = calculate_sample(bsample)
    bmean, bsigma = find_parameters(csample)
    edf = calculate_edf(csample, 100)
    bdelta = calculate_delta_bootstrap(bmean, bsigma, edf)
    if bdelta > delta_kolm:
        k += 1
        
    pbar.set_description(f"Bootstrap критерия Колмогорова p-value: {k / N}")

print('p-value по критерию Колмогорова:', k / N)

Bootstrap критерия Колмогорова p-value: 0.238: 100%|██████████| 50000/50000 [44:56<00:00, 18.54it/s]    

p-value по критерию Колмогорова: 0.238
